# Nearest neighbor search (NNS)

## Abstract
- 点群の最近傍探索(NNS)について紹介する。
- NNS手法として、以下を紹介する。
    - kNN (k Nearest Neighbor)
    - Radius Nearest Neighbor

## Introduction
NNSは点群Aの点から距離的に近い点群Bの点(以下、近傍点)を探し出すことができる。点群では、各点は近傍点から情報を取得もしくは推定することが多いため、NNSは非常に重要である。例として以下が挙げられる。
- 点aのhandcrafted特徴や法線等を求める場合にaの近傍点から周辺の情報を推定し、算出した推定値をaに割り当てるため、NNSが使われる。
- 深層学習のモデルでは、点の近くにある点の特徴を畳み込み機構に入力するため、NNSを利用する。

本セクションでは、XYZ空間上でNNSのチュートリアルを行う。尚、NNSはXYZ空間上だけでなく特徴量空間(例:4次元空間)でも利用可能であるが、本セクションでは説明を簡易にするためにXYZ空間上のチュートリアルのみを扱う。

本チュートリアルでは以下の手法を説明する。
- kNN (k Nearest Neighbor)
- Radius Nearest Neighbor

In [1]:
%load_ext autoreload
%autoreload 2

## kNN (k Nearest Neighbor)
kNNは点の近傍点をk個探すNNSである。チュートリアルコードでは$N$個の点を持つ点群Aの各点の近傍点を点群Bから$k$個探し、$N \times k$サイズの近傍点のインデックス配列を得ることができる。

本subsectionで使用するコードは以下の通り。

In [2]:
# for kNN
import numpy as np
from tutlibs.nns import k_nearest_neighbors
from tutlibs.operator import gather

# for description
from tutlibs.io import Points as io
from tutlibs.utils import single_color
from tutlibs.visualization import JupyterVisualizer as jv
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# load a point cloud data.
xyz, _, _ = io.read("../data/bunny_pc.ply")

# define point cloud A and B.
B = xyz
A = xyz[::500]

# kNN
k = 10
idx, dist = k_nearest_neighbors(A, B, k) # shape: (N, k)
knn_points = gather(B, idx) # shape: (N, k, 3)
knn_points = knn_points.reshape(len(A)*k, 3) # shape: (N*k, 3)

# visualize results.
obj_B = jv.point(B, single_color("#0000ff", len(B)))
obj_knn_points = jv.point(knn_points, single_color("#00ff00", len(knn_points)))
obj_A = jv.point(A, single_color("#ff0000", len(A)))
jv.display([obj_B, obj_knn_points, obj_A])

Output()

上記のコードの出力では、点群A(赤)の近傍点(緑)を点群B(青)から$k$個を取得している。尚、上記例では視覚化によって赤点の下に緑点が隠れているため、上記出力では各点の近傍点の数は赤点の数+緑点の数=$k$であることに注意する必要がある。
次に、kNNの処理を行っているk_nearest_neighbors関数の中身を以下のコードで確認する。

In [4]:
print(inspect.getsource(k_nearest_neighbors))

def k_nearest_neighbors(coords1: np.ndarray, coords2: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    """Compute k nearest neighbors between coords1 and coords2.

    Args:
        coords1: coordinates of centroid points (N, C)
        coords2: coordinates of all points (M, C)
        k: number of nearest neighbors

    Returns:
        idxs: indices of k nearest neighbors (N, k)
        square distances: square distance for kNN (N, k)
    """

    # compute distances between coords1 and coords2.
    point_pairwise_distances = square_distance(coords1, coords2) # shape: (N, M)

    # sort the distances between two points in order of closeness and get top-k indices.
    idxs = np.argsort(point_pairwise_distances, axis=-1)[:, :k] # shape: (N, k)

    # get the distance between two points according to the top-k indices.
    square_dists = np.take_along_axis(point_pairwise_distances, idxs, axis=-1) # shape: (N, k)

    return idxs, square_dists



上記実装をコメントで各段分けて、`処理内容(コメント): 補足`の形式で示す(より細かい説明もしくは補足は段落を下げて示す)。

$N$個の点を含む点群`coords1`と$M$個の点を含む点群`coords2`、近傍点の数$k$を引数とする(上記の説明では、`coords1`が点群A、`coords2`が点群Bに相当)。返り値が近傍点の配列、近傍点との距離を入れた配列である場合、以下の処理がなされる。

1. `compute distances between coords1 and coords2.`: 近傍点をソートで求めるため、`coords1`の各点と`coords2`の各点の相対距離を計算。
2. `sort the distances between two points in order of closeness and get top-k index array.`: インデックスは`coords2`の点群のインデックスである。
3. (option) `# get the distance between two points according to the top-k indices.`: 相対距離をkNN後の処理で利用することがあるため、本実装ではこの様な処理をとっている。


## Radius Nearest Neighbor
Radius nearest neighborsは点からの距離がしきい値$r$未満の近傍点を探すNNSである。チュートリアルコードでは、(1)点群Aの各点の近傍点を点群Bから探して近傍点のインデックス配列を作成し、(2)点間距離がしきい値$r$未満の近傍点であることを示すマスクを取得できる。

本subsectionで使用するコードは以下の通り。

In [5]:
# for kNN
import numpy as np
from tutlibs.nns import radius_nearest_neighbors
from tutlibs.operator import gather

# for description
from tutlibs.io import Points as io
from tutlibs.utils import single_color
from tutlibs.visualization import JupyterVisualizer as jv
import inspect

In [7]:
# load a point cloud data.
xyz, _, _ = io.read("../data/bunny_pc.ply")

# define point cloud A and B.
B = xyz
A = xyz[::500]

# Radius NN
k = 64
r = 0.05
idx, dist, mask = radius_nearest_neighbors(A, B, r, k) # shape: (N, k)
nn_points = gather(B, idx) # shape: (N, k, 3)
rnn_points = nn_points[mask] # shape: (number of neighbors, 3)

# visualize results.
obj_B = jv.point(B, single_color("#0000ff", len(B)))
obj_rnn_points = jv.point(rnn_points, single_color("#00ff00", len(rnn_points)))
obj_A = jv.point(A, single_color("#ff0000", len(A)))
jv.display([obj_B, obj_rnn_points, obj_A])


Output()

上記のコードの出力では、点群A(赤)の各点aは近傍点(緑)をaから距離しきい値$r$範囲内の点群B(青)から取得している。尚、上記例では視覚化によって赤点の下に緑点が隠れているため、上記出力では各点の近傍点の数は赤点の数+緑点の数=近傍点の数であることに注意する必要がある。
次に、kNNの処理を行っているk_nearest_neighbors関数の中身を以下のコードで確認する。

In [8]:
print(inspect.getsource(radius_nearest_neighbors))

def radius_nearest_neighbors(coords1: np.ndarray, coords2: np.ndarray, r: float, k: int = 32) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Compute radius nearest neighbors between coords1 and coords2.

    Args:
        coords1: coordinates of centroid points (N, C)
        coords2: coordinates of all points (M, C)
        r: radius
        k: number of nearest neighbors

    Returns:
        idxs: indices of k nearest neighbors (N, k)
        square distances: square distance for kNN (N, k)
        mask : radius mask (bool) for idxs an distance (N, k)
    """

    # compute kNN.
    idxs, square_dists = k_nearest_neighbors(coords1, coords2, k)

    # get radius nearest neighbors mask.
    radius_mask = square_dists < r**2

    return idxs, square_dists, radius_mask



上記実装をコメントで各段分けて、`処理内容(コメント): 補足`の形式で示す(より細かい説明もしくは補足は段落を下げて示す)。

$N$個の点を含む点群`coords1`と$M$個の点を含む点群`coords2`、距離しきい値`r`、近傍点の最大数$k$を引数とする(上記の説明では、`coords1`が点群A、`coords2`が点群Bに相当)。返り値が近傍点の配列、近傍点との距離を入れた配列、距離しきい値に収まっているか示すマスクである場合、以下の処理がなされる。

1. `compute kNN.`: kNNを用いて近傍点のインデックスと点間の距離配列を取得する。
2. `get radius nearest neighbors mask.`: 距離配列を用いて、`r`内に存在する点であるかを示すマスクを取得する。
